In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.features =16
        # encoder
        self.enc1 = nn.Linear(in_features=3072, out_features=128)
        self.enc2 = nn.Linear(in_features=128, out_features=self.features * 2)

        # decoder
        self.dec1 = nn.Linear(in_features=self.features, out_features=128)
        self.dec2 = nn.Linear(in_features=128, out_features=3072)

    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, self.features)
        # get `mu` and `log_var`
        mu = x[:, 0, :]  # the first feature values as mean
        log_var = x[:, 1, :]  # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)

        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = torch.sigmoid(self.dec2(x))
        return reconstruction, mu, log_var

In [4]:
    # 1. In simple terms, a variational autoencoder is a probabilistic version of autoencoders.
    # 2. VAE to be able to sample from the latent vector (z) space to generate new data, which is not possible with vanilla autoencoders.
    # 3. Each latent variable z that is generated from the input will now represent a probability distribution 
    #    (or what we call the posterior distribution denoted as p(z∣x))
    # 4. All we need to do is find the posterior p(z∣x) or solve the inference problem.
    # 5. In fact, the encoder will try to approximate the posterior by computing another distribution q(z∣x) known as the variational posterior.
    #!6 Probability distribution is fully characterized by its parameters. In the case of the Gaussian, these are the mean μ and the standard deviation σ
    #!7. Decoder will receive the distribution parameters and try to reconstruct the input x. However you cannot backpropagate through a sampling operation.
    #!8. 
import torch.optim as optim
import torch.nn as nn
    
#Train a variational autoencoder 
def elbo(reconstructed, input, mu, logvar):    
    bce_loss = nn.BCELoss(reduction='sum')
    BCE = bce_loss(reconstructed, input)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

IndentationError: expected an indented block after function definition on line 14 (284544665.py, line 15)

In [5]:
# Reparameterization trick
def reparameterize(self, mu, log_var):
        """
        In order to generate samples from the encoder and pass them to the decoder, we also need to utilize the reparameterization trick. 
        Don’t forget that we need to be able to run the backward pass during training.
        
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5 * log_var)  # standard deviation
        eps = torch.randn_like(std)  # generate sample of the same size
        sample = mu + (eps * std)  # sampling as if coming from the input space
        return sample

In [ ]:
# Analysis of loss terms
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
import torch.optim as optim

def train(model,training_data):

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss(reduction='sum')

    running_loss = 0.0

    for epoch in range(1):  # loop over the dataset multiple times

        for i, data in enumerate(training_data, 0):
            inputs, _ = data
            inputs = inputs.view(inputs.size(0), -1)

            optimizer.zero_grad()
            reconstruction, mu, logvar = model(inputs)
            bce_loss = criterion(reconstruction, inputs)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    PATH = './cifar_net.pth'
    torch.save(model.state_dict(), PATH)

    print('Finished Training')